# Analysis

In [ ]:
import os
import pickle
import sys
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import proplot as pplt
import scipy.fft
from tqdm.notebook import trange
from ipywidgets import interact
from ipywidgets import widgets

sys.path.append("/home/46h/repo/psdist/")
import psdist as ps
import psdist.visualization as psv

# Local
import lebedev_bogacz as LB
from tunes import compute_tunes_phase_diff
from tunes import compute_tunes_fft
from tunes import fft

In [ ]:
pplt.rc["colorbar.width"] = "1.2em"
pplt.rc["cmap.discrete"] = False
pplt.rc["cmap.sequential"] = "viridis"
pplt.rc["cycle"] = "538"
pplt.rc["figure.facecolor"] = "white"
pplt.rc["grid"] = False

In [ ]:
save = False

## Load data 

In [ ]:
timestamp = 230929115448
datadir = f"../../data_output/sns_ring/track_matched_danilov/{timestamp}/"    
os.listdir(datadir)

In [ ]:
if save:
    outdir = f"../../figures/sns_ring/track_bunch/{timestamp}/"
    if not os.path.isdir(outdir):
        os.makedirs(outdir)

def save_figure(filename):
    if not save:
        return
    filename = os.path.join(outdir, filename)
    plt.savefig(filename)

## Linear lattice analysis 

### Transfer matrix

In [ ]:
file = open(os.path.join(datadir, "lattice_params_4d.pkl"), "rb")
lattice_params_4d = pickle.load(file, encoding="latin1")
file.close()

pprint(lattice_params_4d)

### Position-dependent parameters

These might be wrong.... they were computed by analyzing the transfer matrix starting from each node, rather than propagating the Twiss parameters along the beamline.

In [ ]:
filename = os.path.join(datadir, "lattice_twiss_4d.dat")
twiss_4d = pd.read_table(filename, sep=" ", index_col=0)
twiss_4d.head()

In [ ]:
fig, ax = pplt.subplots(figsize=(6, 2))
_colors = pplt.Cycle("538").by_key()["color"]
ax.plot(twiss_4d["s"], twiss_4d["beta_1x"], label=r"$\beta_{1x}$", color=_colors[0])
ax.plot(twiss_4d["s"], twiss_4d["beta_1y"], label=r"$\beta_{2y}$", color=_colors[0], alpha=0.2)
ax.plot(twiss_4d["s"], twiss_4d["beta_2x"], label=r"$\beta_{1x}$", color=_colors[1], alpha=0.2)
ax.plot(twiss_4d["s"], twiss_4d["beta_2y"], label=r"$\beta_{2y}$", color=_colors[1])
ax.legend(loc="right", ncols=1)
ax.format(xlabel="Position [m]", ylabel="[m/rad]")
save_figure("twiss_4d")
plt.show()

In [ ]:
filename = os.path.join(datadir, "lattice_dispersion.dat")
dispersion = pd.read_table(filename, sep=" ", index_col=0)

fig, ax = pplt.subplots(figsize=(6, 2))
ax.plot(dispersion["s"], dispersion["disp_x"], label=r"$D_x$")
ax.plot(dispersion["s"], dispersion["disp_y"], label=r"$D_y$")
ax.legend(loc="right", ncols=1)
ax.format(xlabel="Position [m]", ylabel="Dispersion")
save_figure("dispersion")
plt.show()

## Scalar history 

In [ ]:
history = pd.read_csv(os.path.join(datadir, "history.dat"))
history.columns

In [ ]:
fig, ax = pplt.subplots(figsize=(6, 2))
ax.plot(1000.0 * history.loc[:, "x_rms"].values, label="x")
ax.plot(1000.0 * history.loc[:, "y_rms"].values, label="y")
ax.format(ymin=0.0, ymax=(1.25 * ax.get_ylim()[1]))
ax.format(xlabel="Turn", ylabel="RMS size [mm]")
ax.legend(loc="r", ncols=1)
save_figure("rms")
plt.show()

In [ ]:
colors = pplt.Cycle("538").by_key()["color"]

fig, ax = pplt.subplots(figsize=(6, 2))
ax.plot(1.00e+06 * history.loc[:, "eps_1"].values, label=r"$\varepsilon_1$", color=colors[0], ls="-")
ax.plot(1.00e+06 * history.loc[:, "eps_2"].values, label=r"$\varepsilon_2$", color=colors[1], ls="-")
ax.plot(1.00e+06 * history.loc[:, "eps_x"].values, label=r"$\varepsilon_x$", color=colors[0], ls=":")
ax.plot(1.00e+06 * history.loc[:, "eps_y"].values, label=r"$\varepsilon_y$", color=colors[1], ls=":")
ax.format(xlabel="Turn", ylabel="Emittance [mm mrad]")
ax.legend(loc="r", ncols=1)
save_figure("emittance")
plt.show()

### Moments

In [ ]:
for i in range(4):
    for j in range(i + 1):
        key = f"cov_{j}-{i}"
        fig, ax = pplt.subplots(figsize=(5.0, 2.0))
        ax.plot(1.00e+06 * history.loc[:, key], color="black")
        _dims = ["x", "x'", "y", "y'"]
        ylabel = r"$\langle {} {} \rangle$".format(_dims[j], _dims[i])
        ax.format(xlabel="Turn", ylabel=ylabel)
        plt.show()

## Phase space coordinates

Load the bunch at a few turns.

In [ ]:
filenames = os.listdir(datadir)
filenames = [f for f in filenames if f.startswith("bunch")]
filenames = sorted(filenames)
filenames = [os.path.join(datadir, f) for f in filenames]

bunches = []
for filename in filenames:
    X = np.loadtxt(
        filename, 
        comments="%",
        usecols=range(6), 
    )    
    X = X[:, :4]
    X = X * 1000.0
    bunches.append(np.copy(X))

In [ ]:
dims = ["x", "xp", "y", "yp", "z", "dE"]

psv.cloud.proj2d_interactive_slice(
    data=bunches,
    dims=dims,
    offset=1.0,
    options=dict(profiles=True)
)

Compute the bunch normalization matrix $W$, whose inverse results in $\Sigma = \text{diag}(\varepsilon_1, \varepsilon_1, \varepsilon_2, \varepsilon_2)$, where $\varepsilon_{j}$ is the intrinsic emittance of mode $j$. We compute this matrix for the first bunch, assuming the phase space distriubiton is approximately unchanged on subsequent turns.

In [ ]:
X = np.copy(bunches[0])
W = LB.get_bunch_normalization_matrix(X)
Winv = np.linalg.inv(W)

## Small bunch

In [ ]:
filenames = os.listdir(datadir)
filenames = [f for f in filenames if f.startswith("smallbunch")]
filenames = sorted(filenames)
filenames = [os.path.join(datadir, f) for f in filenames]

smallbunches = []
for filename in filenames:
    X = np.load(filename)
    X = X[:, :4]
    X = X * 1000.0
    smallbunches.append(X)
smallbunches = np.array(smallbunches)
smallbunches.shape

In [ ]:
smallbunches_n = np.zeros(smallbunches.shape)
for i in range(smallbunches_n.shape[0]):
    smallbunches_n[i, :, :4] = ps.cloud.transform_linear(smallbunches[i, :, :4], Winv)

In [ ]:
turn_start = 0
turn_stop = 10
tunes = compute_tunes_phase_diff(smallbunches_n[turn_start : turn_stop + 1])


xmin = ymin = 0.0
xmax = ymax = 0.25
limits = [(xmin, xmax), (ymin, ymax)]

grid = psv.JointGrid()
grid.plot_cloud(
    tunes, 
    bins=75,
    limits=limits,
    cmap="blues",
)
grid.ax.format(xlim=limits[0], ylim=limits[1])
grid.ax.format(xlabel=r"$\nu_1$", ylabel=r"$\nu_2$")
plt.show()

In [ ]:
@interact(
    part=(0, len(smallbunches[0]) - 1),
    norm=False,
    tmax=widgets.IntSlider(min=0, max=len(smallbunches), value=len(smallbunches)),
)
def update(part, norm, tmax):
    coords = smallbunches_n if norm else smallbunches
    signals = []
    for axis in [0, 2]:
        signal = coords[:, part, axis]
        signals.append(signal)
        
    fig, axs = pplt.subplots(figsize=(9.0, 2.5), nrows=2, ncols=2, spany=False, sharey=False, sharex=False)
    for i, (dim, signal) in enumerate(zip(["x", "y"], signals)):
        freq, amp = fft(signal)
        amp = amp / np.max(amp)
        axs[i, 0].plot(freq, amp, label=dim, color="black")
        axs[i, 1].plot(signal[:tmax], label=dim, color="black")
    axs[0, 0].format(ylabel="Horizontal")
    axs[1, 0].format(ylabel="Vertical")
    axs[1, 0].format(xlabel="Freqency")
    axs[1, 1].format(xlabel="Turn")
    axs[:, 0].format(xlim=(0.0, 0.5), ylim=(-0.02, 1.1))
    plt.show()

In [ ]:
tunes_fft = compute_tunes_fft(smallbunches_n)

In [ ]:
fig, axs = pplt.subplots(ncols=2)
for i, ax in enumerate(axs):
    psv.cloud.plot2d(
        [tunes, tunes_fft][i], 
        ax=ax, 
        bins=50,
        limits=limits,
        mask=False,
        colorbar=True,
    )
axs.format(toplabels=["Phase diff", "FFT"])
axs.format(xlim=limits[0], ylim=limits[1])
axs.format(xlabel=r"$\nu_1$", ylabel=r"$\nu_2$")
plt.show()